# AlphaEvolve

- Created: 26 May 2025
- Trying out some experiments as to what can be evolved
- Uses AgentJo to perform experiments

Overview
- Tries to guess the word john, given the input-output description, eval function description and a hint
- Evaluation ranges from ordinal difference between characters of guessed word and ground truth, to absolute score of number of correct letters
- LLM actually can perform better with absolute score, as it tells with certainty whether a letter is correct - smooth continuous eval functions may not be the best

Hints
- Tried with various hints
- Easy: "You are to output john"
- Medium: "You are to generate and output a 4-letter English word starting with j"
- Hard: "You are to generate and output a 4-letter English word"

Results
- Progress is made for all, solving Easy and Medium in 50 tries (Easy solved in 1 try for either eval, Medium solved in 29 tries for continuous eval, 12 tries for absolute eval)
- Hard is difficult as search space is large, and LLMs do not handle letters well.
- Also, lack of stochasticity may lead to sampling same solution again as the worse solution may not appear in the prompt
- Also, tried with adversarial hints, if hint is outright wrong (e.g. output mary), it does not work. 
- If hint is wrong but towards solution (output a girl's name), it can work.

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from agentjo import parse_yaml
import os

from dotenv import load_dotenv
load_dotenv()

True

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4.1',
        # reasoning_effort = 'low',
        temperature = 0.1,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Set up the evaluation function
- eval_abs_distance will give relative distance from input to ground truth (higher score for closer)
- eval_exact_matches will just give the number of letters that match

In [5]:
def eval_relative(inputs, ground_truth):
    """
    Compute the negative sum of absolute distances between two lists of lowercase letters.

    Parameters
    ----------
    inputs : list of str
        Predicted letters, e.g. 'acf'
    ground_truth : list of str
        True letters, e.g.    'bdc'

    Returns
    -------
    int
        Sum of abs(index(inputs[i]) - index(ground_truth[i])) over all i.

    Raises
    ------
    ValueError
        If the two lists are not the same length.
    """
    inputs = list(inputs)
    ground_truth = list(ground_truth)

    if len(inputs) != len(ground_truth):
        raise ValueError("Inputs and ground truth must have the same length.")
    
    total_dist = 0
    for pred, true in zip(inputs, ground_truth):
        # map 'a'→0, 'b'→1, …, 'z'→25 by ord(letter) - ord('a')
        idx_pred = ord(pred) - ord('a')
        idx_true = ord(true) - ord('a')
        total_dist -= abs(idx_pred - idx_true) 

    return total_dist

In [6]:
def eval_exact(inputs, ground_truth):
    """
    Compute the number of positions where the predicted letter equals the true letter.

    Parameters
    ----------
    inputs : str
        Predicted letters, e.g. 'abc'
    ground_truth : list of str
        True letters, e.g.      'adc'

    Returns
    -------
    int
        Count of positions i where inputs[i] == ground_truth[i].

    Raises
    ------
    ValueError
        If the two lists are not the same length.
    """
    inputs = list(inputs)
    ground_truth = list(ground_truth)
    
    if len(inputs) != len(ground_truth):
        raise ValueError("Inputs and ground truth must have the same length.")
    
    correct = 0
    for pred, true in zip(inputs, ground_truth):
        if pred == true:
            correct += 1

    return correct

In [7]:
eval_exact('abc', 'acb')

1

In [8]:
eval_relative('abc', 'acb')

-2

# Set up safe exec loop in a limited sandbox
- Use remote systems like docker if you want higher security

In [9]:
import sys, ast, importlib
from io import StringIO
import re

def safe_exec(user_code: str,
              func_name: str,
              allowed_imports: list,
              func_args=None,
              func_kwargs=None):
    """
    Execute user_code in-process, but only after:
     - auto-importing modules you explicitly whitelisted
     - auto-whitelisting any builtins your code actually uses
    Returns {'result':…, 'stdout':…}
    """
    func_args   = func_args   or []
    func_kwargs = func_kwargs or {}

    # 1) start with an empty safe-builtins dict
    SAFE_BUILTINS = {}

    # 2) Prepare our globals––we'll fill __builtins__ below
    restricted_globals = {}
    restricted_locals  = {}

    # 3) Extract import lines and perform only allowed imports
    code_lines = []
    for line in user_code.splitlines():
        m_imp  = re.match(r'\s*import\s+(.+)', line)
        m_from = re.match(r'\s*from\s+([\w\.]+)\s+import\s+(.+)', line)
        if m_imp:
            for mod in [m.strip() for m in m_imp.group(1).split(',')]:
                if mod in allowed_imports:
                    restricted_globals[mod] = importlib.import_module(mod)
            continue
        elif m_from:
            pkg, names = m_from.group(1), m_from.group(2)
            if pkg in allowed_imports:
                module = importlib.import_module(pkg)
                for name in [n.strip() for n in names.split(',')]:
                    if name == '*':
                        for attr in dir(module):
                            if not attr.startswith('_'):
                                restricted_globals[attr] = getattr(module, attr)
                    elif hasattr(module, name):
                        restricted_globals[name] = getattr(module, name)
            continue
        code_lines.append(line)

    cleaned_code = "\n".join(code_lines)

    # 4) Automatically pull in any builtins your code really uses
    tree = ast.parse(cleaned_code)
    used_names = {node.id for node in ast.walk(tree) if isinstance(node, ast.Name)}
    for name in used_names:
        if name in dir(__builtins__):
            SAFE_BUILTINS[name] = getattr(__builtins__, name)

    # 5) Finalize globals
    restricted_globals['__builtins__'] = SAFE_BUILTINS

    # 6) Capture stdout
    old_stdout = sys.stdout
    sys.stdout  = StringIO()

    try:
        # 7) Exec the user’s code
        exec(cleaned_code, restricted_globals, restricted_locals)

        # 8) Call their function
        fn = restricted_locals.get(func_name)
        if not callable(fn):
            raise NameError(f"Function {func_name!r} is not defined")

        result = fn(*func_args, **func_kwargs)

        # 9) Grab anything printed
        output = sys.stdout.getvalue()
        return {'result': result, 'stdout': output}

    except Exception as e:
        # on error, return None + the exception text
        return {'result': None, 'stdout': str(e)}

    finally:
        sys.stdout = old_stdout

# Set up the input and output format and the hints

In [10]:
input_output_format = '''
Inputs: num_letters (int): Number of letters in the word - fixed at 4
Output: word (str): Output word with exactly num_letters lowercase letters'''

In [11]:
problem_hint_hard = "You are to generate and output a 4-letter English word"

In [25]:
problem_hint_medium = "You are to generate and output a 4-letter English word starting with j"

In [13]:
problem_hint_easy = "You are to output john"

# Do the iterative loop

In [14]:
ground_truth = 'john'

In [15]:
allowed_imports = ['string']

In [16]:
def run_experiment(input_output_format,
                   problem_hint,
                   eval_fn,
                   eval_fn_desc,
                   allowed_imports,
                   ground_truth,
                   num_iter=50):
    """
    Runs the experiment and returns the best-scoring `my_soln` code.

    Args:
      input_output_format (str): Defines the I/O relation
      problem_hint          (str): Guidance for the problem
      eval_fn (function): Evaluation that takes in output and ground truth and returns score
      eval_fn_desc (str): Description of what eval function does
      allowed_imports (list): Allowed import modules
      ground_truth          (list): True output to compare against
      num_iter              (int): How many iterations to try

    Returns:
      best_code (str or None): the code of the best‐scoring solution (highest integer score),
                   or None if every run errored.
    """
    soln = []
    best_score = None
    best_code  = None

    for i in range(num_iter):
        # --- pick top 20 past solutions by integer score (None→−∞)
        def score_key(entry):
            s = entry.get("Result")
            return s if isinstance(s, int) else float("-inf")
        top20 = sorted(soln, key=score_key, reverse=True)[:20]

        # 1) generate a candidate
        res = parse_yaml(
            system_prompt=f"""Generate a python function named my_soln that takes in inputs,
and returns outputs to solve a problem.
Output your analysis about the trend, thoughts about what to try next, and the next code to try.
Maximise the eval function score and use past solutions to figure out the correct approach.
You should actively seek to try out new promising variations from the best program that may maximise the eval function - do not need to exploit, just explore.
The variations should be small and avoid radical changes.
Do not repeat the past solutions.
Avoid any randomness.
Use the problem hint to guide the solution.

Allowed imports (only use these imports in your code): {allowed_imports}
Current best program to modify from: {best_code}
Top 20 past solutions and eval scores to learn general trend: {top20}
Input/output format: {input_output_format}
Problem hint: {problem_hint}
Eval Function Description: {eval_fn_desc}
""",
            user_prompt="",
            output_format={"Analysis": "Analyse past solutions to find a trend, type: str",
                           "Thoughts": "Thoughts on what to try next, type: str",
                           "Code": "Generated python function named my_soln, type: str"},
            llm=llm
        )

        print(f'''\nIteration {i+1}
Analysis: {res['Analysis']}
Thoughts: {res['Thoughts']}
Code: {res['Code']}''')

        # 2) safely exec it
        out = safe_exec(
            user_code=res['Code'],
            func_name="my_soln",
            allowed_imports=allowed_imports,
            func_args=[4],     # ← adjust your input here
        )

        # 3) interpret the result
        if out["result"] is None:
            score = None   # error case, ignore
        else:
            score = eval_fn(out["result"], ground_truth)

        # 4) record it
        soln.append({"Code": res['Code'], "Result": score})
        print(f"Result: {score}")

        # 5) update best if it's a valid integer and better than before
        if isinstance(score, int):
            if best_score is None or score > best_score:
                best_score = score
                best_code  = res['Code']

        # 6) early exit if perfect
        if out["result"] == ground_truth:
            print(f"Solved exactly on iteration {i+1}")
            break

    return best_code

# Use a more expressive loss function
- LLMs unable to tell ordinal difference well between letters
- Lack of stochasticity can lead to generating the same code changes although it does not improve the score - since it does not appear in prompt
- Solves easy (1 try) and medium (29 tries)
- Unable to solve hard within 50 tries
- If problem is sufficiently unbounded, can be difficult for LLM to approach the solution

In [17]:
print('Hint given:', problem_hint_easy)
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = problem_hint_easy,
               eval_fn = eval_relative,
               eval_fn_desc = "Sums the negative difference between word and ground truth of absolute ordinal position of each letter at each position. Top score is 0",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth)

Hint given: You are to output john

Iteration 1
Analysis: There are no past solutions to analyze, but the problem hint says to output "john" and the eval function rewards matching the ground truth letter-by-letter. The input num_letters is fixed at 4, so the output should be a 4-letter word, specifically "john".

Thoughts: The best initial approach is to return "john" directly, as the hint suggests. For future variations, if the eval score is not perfect, I could try adjusting the case or check for off-by-one errors in the letters.

Code: def my_soln(num_letters):
    return "john"

Result: 0
Solved exactly on iteration 1


In [27]:
print('Hint given:', problem_hint_medium)
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = problem_hint_medium,
               eval_fn = eval_relative,
               eval_fn_desc = "Sums the negative difference between word and ground truth of absolute ordinal position of each letter at each position. Top score is 0",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth)

Hint given: You are to generate and output a 4-letter English word starting with j

Iteration 1
Analysis: There are no past solutions to analyze, but the problem hint specifies that the output should be a 4-letter English word starting with 'j'. The eval function rewards minimizing the absolute ordinal difference between the output word and a ground truth word, letter by letter. Since the ground truth is unknown, a reasonable first attempt is to pick a common 4-letter English word starting with 'j', such as "jump". This will provide a baseline for future iterations.

Thoughts: Since the ground truth is unknown, the best approach is to start with a valid 4-letter English word starting with 'j'. In future attempts, I can try other common words like "jazz", "just", or "joke" to see if the eval score improves. For now, I will use "jump" as the initial guess.

Code: def my_soln(num_letters):
    # Output a 4-letter English word starting with 'j'
    return "jump"

Result: -13

Iteration 2
A

In [19]:
print('Hint given:', problem_hint_hard)
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = problem_hint_hard,
               eval_fn = eval_relative,
               eval_fn_desc = "Sums the negative difference between word and ground truth of absolute ordinal position of each letter at each position. Top score is 0",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth)

Hint given: You are to generate and output a 4-letter English word

Iteration 1
Analysis: There are no past solutions to analyze yet. The problem requires generating a 4-letter English word using only lowercase letters. The eval function rewards words that are closer in absolute ordinal position to a ground truth word, letter by letter. Since there is no information about the ground truth word, the best initial approach is to try a common 4-letter English word as a baseline.

Thoughts: Since the eval function compares the generated word to a ground truth word, starting with a common 4-letter word like "word" is a reasonable baseline. In the next iteration, I can try other common 4-letter words or vary one letter at a time to see if the score improves.

Code: def my_soln(num_letters):
    # Output a common 4-letter English word as a baseline
    return "word"

Result: -33

Iteration 2
Analysis: The previous solution simply returned the word "word", which is a common 4-letter English wor

# Using a harder eval function
- Solves easy (1 try) and medium (12 tries)
- Unable to solve hard within 50 tries
- If problem is sufficiently unbounded, can be difficult for LLM to approach the solution

In [20]:
print('Hint given:', problem_hint_easy)
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = problem_hint_easy,
               eval_fn = eval_exact,
               eval_fn_desc = "Counts the number of correct letters in the correct position when compared to the ground truth",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth)

Hint given: You are to output john

Iteration 1
Analysis: There are no past solutions to analyze. The problem hint says to output "john" and the eval function counts the number of correct letters in the correct position compared to the ground truth. The input num_letters is always 4, so the output should be a 4-letter word, and the hint suggests "john" is the correct answer.

Thoughts: The most promising approach is to return "john" directly when num_letters is 4. For future variations, if the problem changes to allow different values of num_letters, we could consider slicing or repeating "john" to fit the required length, but for now, we will stick to the direct solution.

Code: def my_soln(num_letters):
    return "john"

Result: 4
Solved exactly on iteration 1


In [26]:
print('Hint given:', problem_hint_medium)
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = problem_hint_medium,
               eval_fn = eval_exact,
               eval_fn_desc = "Counts the number of correct letters in the correct position when compared to the ground truth",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth)

Hint given: You are to generate and output a 4-letter English word starting with j

Iteration 1
Analysis: There are no past solutions to analyze, but the problem hint specifies that the output should be a 4-letter English word starting with 'j'. The eval function rewards correct letters in the correct position. Since the number of letters is fixed at 4, and the word must start with 'j', a good initial guess is to use a common 4-letter English word starting with 'j', such as "jump".

Thoughts: The next step is to try a common 4-letter English word starting with 'j'. If this does not maximize the eval score, future variations could involve trying other common 4-letter words starting with 'j' (e.g., "jazz", "jury", "just", etc.). For now, I will try "jump" as the output.

Code: def my_soln(num_letters):
    # Output a common 4-letter English word starting with 'j'
    return "jump"

Result: 1

Iteration 2
Analysis: The only past solution outputs "jump", a common 4-letter English word star

In [22]:
print('Hint given:', problem_hint_hard)
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = problem_hint_hard,
               eval_fn = eval_exact,
               eval_fn_desc = "Counts the number of correct letters in the correct position when compared to the ground truth",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth)

Hint given: You are to generate and output a 4-letter English word

Iteration 1
Analysis: There are no past solutions to analyze, but the problem requires generating a 4-letter English word using only lowercase letters. The eval function rewards correct letters in the correct position compared to a ground truth word. Since there is no information about the ground truth, starting with a common 4-letter English word is a reasonable first step.

Thoughts: Since the problem asks for a 4-letter English word, I will start with a very common word such as "word". In the next iteration, I could try other common 4-letter words or use letter frequency analysis to maximize the chance of matching the ground truth.

Code: def my_soln(num_letters):
    # Output a common 4-letter English word
    return "word"

Result: 1

Iteration 2
Analysis: The only past solution outputs the word "word" for any input, which is a valid 4-letter English word. The eval function rewards matching letters in the correct 

# Giving a wrong hint
- Solves within 50 tries (when asked to output a girl's name)

In [24]:
print('Hint given:', "Output a girl's name")
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = "Output a girl's name",
               eval_fn = eval_exact,
               eval_fn_desc = "Counts the number of correct letters in the correct position when compared to the ground truth",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth,
               num_iter = 50)

Hint given: Output a girl's name

Iteration 1
Analysis: There are no past solutions to analyze. The problem requires outputting a 4-letter girl's name. The eval function rewards correct letters in the correct position compared to the ground truth. Since there is no information about the ground truth, a reasonable first attempt is to use a common 4-letter girl's name.

Thoughts: I will try the name "anna", which is a common 4-letter girl's name. If this does not score well, I will try other common names in subsequent attempts, making small variations.

Code: def my_soln(num_letters):
    # Output a common 4-letter girl's name
    return "anna"

Result: 0

Iteration 2
Analysis: The only past solution outputs "anna" for a 4-letter girl's name, which is a common choice. The eval score is 0, indicating that "anna" is not the correct answer. The problem asks for a 4-letter girl's name, and the eval function compares the output to a ground truth by counting correct letters in the correct posi

In [28]:
print('Hint given:', "Output mary")
soln = run_experiment(input_output_format = input_output_format,
               problem_hint = "Output mary",
               eval_fn = eval_exact,
               eval_fn_desc = "Counts the number of correct letters in the correct position when compared to the ground truth",
               allowed_imports = allowed_imports,
               ground_truth = ground_truth,
               num_iter = 10)

Hint given: Output mary

Iteration 1
Analysis: There are no past solutions to analyze. The problem hint says to output "mary" and the eval function counts the number of correct letters in the correct position compared to the ground truth. The input num_letters is fixed at 4, so the output should be a 4-letter word.

Thoughts: Since the hint says to output "mary", the most promising approach is to return "mary" when num_letters is 4. This should maximize the eval score. For the next step, if this doesn't achieve the maximum score, I could try variations like returning other 4-letter words, but "mary" is the most likely correct answer based on the hint.

Code: def my_soln(num_letters):
    return "mary"

Result: 0

Iteration 2
Analysis: The only past solution returns the string "mary" regardless of the input, and it achieved a score of 0. The problem hint is to output "mary", and the input is always num_letters=4. The eval function counts correct letters in the correct position compared 